<a id="representation"></a>
# Representation learning revisited

After gaining insight about the architectures and procedures that make training of deep neural networks possible, let's get back to the topic of representations. Originally we motivated the usage of deep learning with two things: 
1. Superior performance

That is the ability to surpass traditional models in relevant measures.

2. End-to-end learning

The ability to forego much of the manual feature engineering process necessary for classical models.

The representation ability of deep learning models is in strong connection with the latter. 
Let us examine this in detail!

## Learned representations in "shallow" networks: word2vec
**"Don't count, predict!"**


<a href="http://drive.google.com/uc?export=view&id=1uu00eAJi3-3tH2Iz8IMP9vz3iWtgdmqM"><img src="https://drive.google.com/uc?export=view&id=1XVg5kBnHm3N5NrnRvTgwKf6Pk_0-TM7u" width=600 heigth=600></a>


With the publication of "Distributed Representations of Words and Phrases and their Compositionality" by [Mikolov et al. 2013](https://arxiv.org/pdf/1310.4546.pdf) a _huge_ shift occured in the NLP community, that led away from frequency based methods and introdiced the usage of prediction based methods for the generation of efficient language models. (First at word level.)

### Schematics

<a href="https://raw.githubusercontent.com/rohan-varma/paper-analysis/master/word2vec-papers/models.png"><img src="https://drive.google.com/uc?export=view&id=1ZUwSGzEj4EWHCEabrrvmXadfRsdRF7_f" width=600 heigth=600></a>

<a href="https://i.stack.imgur.com/igSuE.png"><img src="https://drive.google.com/uc?export=view&id=1R5hI-k4Zu8P6bE3oGgmJ5cMpDYshU7ML" width=300 heigth=300></a>

(Important to note that the invention of "hierarchic softmax" came from this research, since the many $v$ vocabulary width layers were consuming extreme amout of computation (by 2013 standards) for a vocabulary of 300k. Based on this there are CPU programmable efficient implementations of word2vec "out of the box", like in [Gensim](https://radimrehurek.com/gensim/models/word2vec.html).  

### Advantage

The real advantage of these "word embeddings" (which became the workhorse of NLP eversince) was not that they were useful in predicting the next words "autocomplete style" (as we have seen before in our training), but much more as general dense vector representations, "embeddings" of words. The main breakthrough of Mikolov et al. was to discover the deep structure that the vectorspaces exhibit after training!


<a href="http://drive.google.com/uc?export=view&id=1heogQhMfvtiOSfPtKvmc2OtyGadAsOmd"><img src="https://drive.google.com/uc?export=view&id=1HbQDy8orwiRH7SiaJjE5Gu5g99iavZa5" width=600 heigth=600></a>


A good analysis of this topic can be found in [Marek Rei's blogpost](http://www.marekrei.com/blog/dont-count-predict/).

(Naturally, progress did not stop here, you can read up on successive generations of vector embedding for NLP [here](https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a).)

### Why can this be?

For the model to solve the prediction task effectively, it has to come up with a representation that captures the salient features of the data in the most copact way, that is, it lossfully memorizes and compresses the data, during which it captures it's main features.

**It turns out that the decisive advantage of deep learning based methods is exactly this: the "byproduct" of learning hierarchic, meaningful features during training.** Throughout this class, we will examine the effects and possibilities arising from this. 

Remember:
**Representation is everything!**

## New distance-metric oriented supervised tasks and loss functions

The growing recognition that models trained on particular supervised tasks can learn representations useful for a wide range of different purposes led to a research into tasks and objectives that are more conducive to learning good distributed representations.

One of the explicit goals of these objectives is to learn useful similarity distance metrics over the domain.

Image similarity recognition (in particular face recognition) was the most important problem that motivated this research, but in the last few years the methods have been heavily used in other domains as well, e.g. for voice verification/identification. The most important characteristics of these tasks and objectives is that they

- work with coarse grained ranking data about similarity, crucially, they can be used on labeled/classification data sets where the only available similarity information comes from class membership (examples belong to the same or different class)
- produce (similarly to Word2Vec) _sparse distributed representations_ / _embeddings_ of the input 
- try to measure and maximize the quality of the distance-based similarity metric provided by the learned representations
- the objective is typically to keep members of the same class close to each other and distant from those of other classes (in the embedding space)

### Contrastive loss (2005)

The first step in this direction was the introduction of __contrastive loss__ by Hadsell et al. in 2005 ([Dimensionality Reduction by Learning an Invariant Mapping](http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf)), which is a loss function for __pairs of examples__ $\langle \mathbf x_1, \mathbf x_2\rangle$:

$$\mathcal L_{\mathrm{Contrastive}} = (1-y)\frac{1}{2}D(\mathbf x_1, \mathbf x_2)^2 + y \frac{1}{2}(\max(0, m-D(\mathbf x_1, \mathbf x_2))^2$$

where $y$ is 0 if the two examples are similar and 1 if not, and D is a distance metric between the embeddings, e.g.,  Euclidean or cosine distance. The intention is obvious: bring similar examples closer and separate dissimilar examples by an $m$ margin.  

The architecture used with this kind of loss is typically a so-called "Siamese network", in which embeddings are produced with the same network topology and weights for both examples:

<a href="https://miro.medium.com/max/841/1*fUY-bpGFoUMWBkh_rychCQ.jpeg"><img src="https://drive.google.com/uc?export=view&id=1GnyGX-FsC_29NLGYEyJRNZYVi6kPabpj" width="350px"></a>

(Image source: [Siamese Networks for Visual Tracking](https://medium.com/intel-student-ambassadors/siamese-networks-for-visual-tracking-96262eaaba77))

__Challenges__

Perhaps the most important challenge for contrastive loss is that incentives the model to keep examples from the same class very close to each other, so it is prone to concentrate the classes into small areas of the space. As a consequence, the learned representations lack distinctions within the classes.

###  Triplet loss (2010)

In many respects a follow-up to contastive loss, triplet loss tries to solve the class concentration problem of contrastive loss by considering data point _triplets_ and enforcing only the correct _relative distances_. The inputs are $\langle A, P, N\rangle$ triplets containing a so called $A$ _anchor example_, and a $P$ positive and an $N$ negative example, of which the former belongs to the same class as the anchor, while the second to a different one.

The objective is to keep an (SVM-like) positive margin between the distance/similarity of the anchor and the positive and the anchor and the negative example:

$$
\mathcal L(A, P, N) = \max(D(A, P) - D(A, N) + margin, 0)
$$

Where the $D(\cdot)$ distance function is commonly Euclidean or cosine distance between the embeddings of the examples.


<a href="https://omoindrot.github.io/assets/triplet_loss/triplet_loss.png"><img src="https://drive.google.com/uc?export=view&id=1w7Yx2yJbTJ7pdaLaUfOtWDNi9BRQLDiz" width="500px"></a>

([Image source](https://omoindrot.github.io/triplet-loss))

__Challenges__

Training with triplet loss is known to be challenging, since the results are dependent on the "difficulty" of the triplets that are fed into to network. As training on all possible triplets is usually unfeasible and would not be desirable anyway, smart and often resource intensive "triplet mining" techniques have to employed to select the "hard" and "semi-hard" triplets in which the negative example is closer to the anchor than the positive or the distance is smaller than the margin:

<a href="https://omoindrot.github.io/assets/triplet_loss/triplets.png"><img src="https://drive.google.com/uc?export=view&id=1Ur_mBDuhDRdSacfarQ9CnhGGCstUJadX" width="400px"></a>

([Image source](https://omoindrot.github.io/triplet-loss))

__See also__

+ A good, TF-based introduction from which the above figures were taken: [Triplet Loss and Online Triplet Mining in TensorFlow](https://omoindrot.github.io/triplet-loss)
+ The classic, original triplet loss paper on image similarity detection: [Chechik et al.: Large Scale Online Learning of Image Similarity Through Ranking (2010)](http://www.jmlr.org/papers/volume11/chechik10a/chechik10a.pdf)
+ Using triplet loss for face recognition: [Schroff et al: FaceNet: A Unified Embedding for Face Recognition and Clustering (2015)](https://arxiv.org/pdf/1503.03832.pdf)

### New "softmax variants"

The training difficulties associated with triplet loss led to the development of new, similarity metric-oriented variants of the traditional softmax--cross entropy objective for classification. Recall that for one-hot encoded classification training data, softmax-cross entropy boils down to a negative log-likelihood objective:

$$\mathcal L_\mathrm{Softmax} = - \log(P(y)) $$

where thinking in terms of the last fully connected layer of a classification network we have further

$$-\log(P(y)) = -\log\left(\frac{\exp(\mathbf w_y \mathbf x + b_y)}{\sum_c \exp(\mathbf w_c \mathbf x + b_c) }\right)$$

where $\mathbf x$ is the embedding and $y$ the correct class of the example in question.

#### Angular softmax (A-softmax, 2018)

Angular softmax is based on the recognition that with normalized weight vectors and zero biases (this if of course an important modification in itself!!) the softmax loss can actually be rewritten as

$$-\log\left(\frac{\exp(\mathbf w_y \mathbf x)}{\sum_c \exp(\mathbf w_c \mathbf x) }\right) = -\log\left(\frac{\exp(\|\mathbf x\| \cos \theta_{\mathbf x, \mathbf w_y })}{\sum_c \exp( \|\mathbf x\| \cos \theta_{\mathbf x, \mathbf w_c })}\right) = -\log\left(\frac{\exp(\cos \theta_{\mathbf x, \mathbf w_y })}{\sum_c \exp( \cos \theta_{\mathbf x, \mathbf w_c })}\right)$$

that is, this modified version requires the angle between the embedding and the correct class's weight vector to be smaller then the angles to those of the other classes. The main idea of angular softmax is to strengthen this requirement to the criterion that $m$ times the angle should still be significantly smaller (introducing an "angular margin"), where $m\geq 2$ is a hyperparameter, so we have

$$
-\log\left(\frac{\exp(\cos (m  \theta_{\mathbf x, \mathbf w_y }))}{\exp(\cos (m \theta_{\mathbf x, \mathbf w_y })) + \sum_{c\neq y}\exp( \cos \theta_{\mathbf x, \mathbf w_c })}\right)
$$

A slight problem is caused by the fact that the cosine function is not monotonic outside the $[0, \pi]$ interval, so $\theta_{\mathbf x, \mathbf w_y}$ here must be restricted to the $[0, \frac{\pi}{m}]$ interval. The problem can be solved by using a cosine derivative which is monotonic in the full $[0, \pi]$, concretely

$$\phi(\theta) = (-1)^k \cos(m \theta) - 2k$$
$$\left(\theta\in \left[\frac{k\pi}{m},\frac{(k+1)\pi}{m}\right], k \in [0, m-1]\right).$$ 

With this change we have the final form which is

$$
\mathcal L_{\mathrm{A-softmax}} = -\log\left(\frac{\exp(\phi (\theta_{\mathbf x, \mathbf w_y }))}{\exp(\phi ( \theta_{\mathbf x, \mathbf w_y })) + \sum_{c\neq y}\exp( \cos \theta_{\mathbf x, \mathbf w_c })}\right).
$$
For further details see the paper in which A-Softmax was introduced: [Huang et al: Angular Softmax for Short-Duration Text-independent Speaker Verification (2018)](https://www.isca-speech.org/archive/Interspeech_2018/pdfs/1545.pdf)

#### Additive margin softmax  (AM-softmax, 2018)

Another softmax variant built on the same basic idea is the so called additive margin softmax loss, which demands that the embedding should be $m$ more similar to the correct class's weight vector than to the others:

$$
\mathcal L_{\mathrm{AM-softmax}} = -\log\left(\frac{\exp(s\cdot\cos (\theta_{\mathbf x, \mathbf w_y }-m))}{\exp(s\cdot \cos ( \theta_{\mathbf x, \mathbf w_y }-m)) + \sum_{c\neq y}\exp(s\cdot\cos \theta_{\mathbf x, \mathbf w_c })}\right)
$$

where $s$ is a scaling hyperparameter. 

See the original paper for further details: [Wang et al: Additive Margin Softmax for Face Verification (2018)](https://arxiv.org/pdf/1801.05599.pdf)

<a id="transfer"></a>
# Transfer learning

- Presupposition till this point, that we build models specificly for the given task and always start training "from scratch".
- Do not take intuitions, insights, models and hyperparameters from one domain and apply it to another one. 
- If generalization of models is strong, should be able to utilize knowledge stored in one learned model in another, smilar case. (The concept of "similar task" is non-obvious!)

Main source for discussion: [Michéle Sebag: Representation Learning, Domain Adaptation and Generative Models with Deep Learning. DeepLearn2018 Summer School, Genova](https://drive.google.com/file/d/1XZjoYuOxdPUPGZFAVw2_9Ob78msCKzNF/view?usp=sharing)

A very good summary can be found at the blog of [Sebastian Ruder](http://ruder.io/transfer-learning/). 

Continuum between Domain Adaptation, Transfer Learning, and Multi-Task learning.

- **Domain Adaptation**: same task, but "domain" for the task different (adapting spam filter from one user to another user)
- **Trasfer learning**: different task (telling apart cars and telling apart animals, or even a classification and a regression), usually different tasks done sequentially
- **Multi-Task learning**: strengthen model representation by giving it multiple different tasks  (with the idea that the tasks at least have something in common)


## Motivation and basics

* Task: classification, or regression
* A source domain source distribution $D_s$ 
* A target domain target distribution $D_t$

**Idea:**
* Source and target are “sufficiently” related
* ... one wants to use source data to improve learning from target data

### Settings

<a href="http://drive.google.com/uc?export=view&id=1nu7qrEhJz0obfwEzcTqB9FbJe4MSHK-T"><img src="https://drive.google.com/uc?export=view&id=1aGxmOevZIK8Zrzm2iOy3T5QGC885Cqr0" width=600 heigth=600></a>
<a href="http://drive.google.com/uc?export=view&id=11neGgvY5dy5BaS34edW7KfjY9uCdmspl"><img src="https://drive.google.com/uc?export=view&id=1eNEreV9KtsQ0DolXt2zzPHix_Va1Ww1X" width=600 heigth=600></a>

### Warning! - Transferring bias

The myth of the "Tank detector":
https://www.gwern.net/Tanks#could-something-like-it-happen

<a href="https://i.ytimg.com/vi/yvuKQkGVjJE/hqdefault.jpg"><img src="https://drive.google.com/uc?export=view&id=1vxqGrvyWaAPNbTeMSnRl9f-gmRf6yvzg" width=400 heigth=400></a>

It can be an "urban legend", but shows that a mere environmental constant in a dataset can becom a a biasing factor, so it is pretty easy to imagine that these biases will hurt performance in a different setting. (As well as again draw attention to the paramount importance of model validation!)

## Some baseline supervised methods:

### Learn source and target domain in union

[J. Huang, A. Smola, A. Gretton, K. M. Borgwardt, and B. Scholk ¨ opf. Correcting sample selection
bias by unlabeled data. In B. Scholk ¨ opf, J. Platt, and T. Hoffman, editors, Advances in Neural
Information Processing Systems 19. MIT Press, Cambridge, MA, 2007](https://papers.nips.cc/paper/3075-correcting-sample-selection-bias-by-unlabeled-data.pdf)

- Default assumption in many learning scenarios: training and test data independently and identically (iid) drawn from the same distribution 
- While the available data have been collected in a biased manner, the test is usually performed over a more general target population
- Training and test data are drawn from different distributions, commonly referred to as sample selection bias
- Covariance shift
- Matching distributions between training and testing sets in feature space
- Account for the difference between Pr(x, y) and Pr′ (x, y) by reweighting the training points such that the means of the training and test points are close (in a reproducing kernel Hilbert space (RKHS))


Since the source has much more data, target can get "oppressed".

###  Label based on source classifier, add that as input, learn new classifier

[H. Daumé, D. Marcu, Domain Adaptation for Statistical Classifiers](https://www.aaai.org/Papers/JAIR/Vol26/JAIR-2603.pdf)


- Situation: labeled out-of-domain data plentiful, but labeled in-domain data scarce
- Statistical formulation of problem in terms of a simple mixture model
- Treat in-domain data as drawn from a mixture of two distributions: a “truly in-domain” distribution and a “general domain” distribution
- Out-of-domain data is treated as if drawn from a mixture of a “truly out-of-domain” distribution and a “general domain” 
- Inference algorithm case based on the technique of conditional expectation maximization


### Learn two classifiers, one for source, one for target and weight their outputs as "votes"

[H. Daumé, D. Marcu, Domain Adaptation for Statistical Classifiers](https://www.aaai.org/Papers/JAIR/Vol26/JAIR-2603.pdf)

### Use source model as prior

Initialize the target classifier with the weights learned from source, during training regularize towards the original weights. 

#### Detour: Catastrophic forgetting

- What if during learning the new task, we don't just acquire new abilities, but "forget" what we have learned so far? 
- No guarantee that updates based on error gradients on the new task do not lead the model weights away from the optimum learned before (or better to say: it is nearly certain it does so...) 
- It is a difficult task to ensure, that learning does not happen with strong degradation of the prior knowledge (what prior knowledge to degrade)?

**[Overcoming catastrophic forgetting in neural networks](https://arxiv.org/abs/1612.00796)**

- One Idea: make the change of weights that were important for solving prior tasks more difficult. 
- We can use a quadratic error term we can try to keep these weights close to their original values. The authors call this "elastic weight consolidation". 

<a href="https://image.slidesharecdn.com/overcomingcatastrophicforgettinginneuralnetwork-170429024916/95/overcoming-catastrophic-forgetting-in-neural-network-9-638.jpg?cb=1493434190"><img src="https://drive.google.com/uc?export=view&id=1Wh7u-waBs_hV2lWcawgjwVwdAHcWOA2q"></a>

An interesting description of "catastrophic forgetting" in NLP context can be found [here](https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting).

### Unsupervised transfer learning - Distance of tasks 

####  Feature augmentation

[H. Daumé, Frustratingly Easy Domain Adaptation In ACL 2007](https://arxiv.org/abs/0907.1815)
- Take each feature in the original problem and make three versions of it: a general version, a source-specific version and a target-specific version. 
- Augmented source data will contain only general and source-specific versions. 
- Augmented target data contains general and target-specific versions 
- Then define a systematic mapping between source and target data as a transformation.
- Distance of tasks" relevant: presume systematic mapping transforming source to target. 

<a href="http://drive.google.com/uc?export=view&id=1lhdJQBM61Dxg8fH0nMbBezJK9qT2oQrf"><img src="https://drive.google.com/uc?export=view&id=1zDTSMLFNzIJ9P6njCIi6UoC4Sjt_tuOw" width=600 heigth=600></a>

For one method see [Baochen Sun, Jiashi Feng, Kate Saenko: Return of Frustratingly Easy
Domain Adaptation. AAAI 2016](https://arxiv.org/abs/1511.05547) quoted by Sebag.

#### Learning of "general" representations

- Can argue, that by learning on a big enough dataset, a model will learn useful "general" features, that can be later on reusable. 
- This is indeed one of the motivation behind teaching autoencoders (large amounts of unlabeled data), or **re-using model representations** from discriminative models - trained eg. on ImageNet.

**Capitalize on the notion, that the deep networks are gradually learning hierarchic representations of features, and only at the last couple layers do they learn a classifier.**

**Thus a common practice to take a trained ConvNet, throw away the last 1-2 layers and train it for a new task - remember, last is a softmax over the classes...**

### How to prevent catastrophic forgetting?

To answer the question properly, we have to think over **where and how the forgetting happens** at all.

Since the topmost (eg. Softmax) layers are initialized randomly, most probably they contribute most to the error, thus it can be argued, that until we get them to "mature" a bit more, they are the main culprits we have to work with, thus there is growing evidence, that **the top layers have to be trained first, then the lower layers can be fine tuned.**

(This makes all the more sense, since the lower the layers, the more basic features they represent, thus most probably they generalize all the more, so it makes no sense to hit them hard with some big modifications at first.)

Enter:

#### Gradual unfreezing

The idea is pretty simple: for the early part of the transfer learning based training do not allow updates on weights for the majority of the network, just **gradually unfreeze** the layers later on.

<a href="https://humboldt-wi.github.io/blog/img/seminar/group4_ULMFiT/Figure_21.png"><img src="https://drive.google.com/uc?export=view&id=1KxNr1UqL_1q7FTNjLv8SXivkuFKB343H" width=65%></a>

See a detailed analysis of Ruder and co.'s UMLFIT [here](https://medium.com/explorations-in-language-and-learning/transfer-learning-in-nlp-2d09c3dfaeb6).

#### Soft version: Differential learning rates

The concept of [differential learning rates](https://blog.slavv.com/differential-learning-rates-59eff5209a4f) is in a sense a soft version of freezing: instead of fixing the lower layers, a smaller learning rate is used for their updates, which - at least in theory - should help them avoid forgetting.

**In practice, though, many times an appropriately small (think: one order of magnitude smaller) global learning rate is sufficient.**

<a href="https://cdn-images-1.medium.com/max/2400/1*BEyoI-p1FGTV0PUoGTxz4Q.png"><img src="https://drive.google.com/uc?export=view&id=1fdjxkQ-MHqM42k5PPU4A-pfmfZmfAu9J" width=70%></a>

#### Verdict is still out

The development of more effective transfer learning methods is far from finished, there are quite [recent papers](https://arxiv.org/abs/1812.01640) which offer a good survey:

<a href="http://drive.google.com/uc?export=view&id=1V03HbhaHSRboTubR5afsMh0G6TTbkMMw"><img src="https://drive.google.com/uc?export=view&id=1PvsLWLN9aWwYFqINCLpG4gymki-eZcUD"  width=55%></a>


##### Detour: Weak supervision - An interesting current solution

[Exploring the Limits of
Weakly Supervised Pretraining](https://research.fb.com/wp-content/uploads/2018/05/exploring_the_limits_of_weakly_supervised_pretraining.pdf)

- Research group at Facebook wanted to teach a discriminative model with exceptionally large dataset, bigger than manual tagging allows
- Decided to capitalize on "weak supervision", that is the noisy "labeling" based on the # tags people give for their Instagram pictures
- Took **3.5 billion (!!!)** Instagram pictures and trained a ConvNet on it. 
- Annotation quality inferior, but the sheer amount of data causes the model to beat the state of the art in transfer scenarios.

This was also a _huge_ implementation challenge, well worth a read [here](https://www.facebook.com/ross.girshick/posts/10160363792300261)  and [here](https://code.facebook.com/posts/1700437286678763/)




## Multi task learning

Basic idea: advantage to force a model to learn multiple tasks concurrently!
It promises, that we generally learn something about the world - model constrained by variety of data. (AGI, rings a bell? :-)


### Specific models

#### "Zero-shot translation"
[Google’s Multilingual Neural Machine Translation System:
Enabling Zero-Shot Translation](https://arxiv.org/pdf/1611.04558.pdf)

Or in a more readable format [here](https://ai.googleblog.com/2016/11/zero-shot-translation-with-googles.html)

- Seq2seq, "normal" encoder-decoder + attention mechanism machine translation

<a href="https://ai2-s2-public.s3.amazonaws.com/figures/2017-08-08/0245d8cbc0a405e553311bd20a01099c9aa11c14/4-Figure1-1.png"><img src="https://drive.google.com/uc?export=view&id=1UjKks5TT-bf45YkoLh3ACipKLFiGJaA-"></a>

- Parallel corpora on _language pairs_
- Only addition, that a separate tag for the given language is put before the input sequence (to inform the model...

**Result:**

1. It learns to translate to language pairs for which _it was never given data_
2. It's inner representation is _language independent_

<a href="https://2.bp.blogspot.com/-AmBczBtfi3Q/WDSB0M3InDI/AAAAAAAABbQ/1U_51u5ynl4FK4L0KOEllfRCq0Oauzy5wCEw/s640/image00.png"><img src="https://drive.google.com/uc?export=view&id=1Nd9tmG6SBlixVpIKyd7crfbPa3sVf7dw"></a>

This result amazed even the researchers, much analysis went into it.

#### Multi-modality

Even more ambitious experiment:  Google ["One model to rule them all"]( https://arxiv.org/abs/1706.05137)

- Single model that yields good results on a number of problems spanning multiple domains: trained concurrently on ImageNet, multiple translation tasks, image captioning (COCO dataset), a speech recognition corpus, and an English parsing task. 
- Model architecture: convolutional layers, an attention mechanism, and sparsely-gated layers
- Each of these computational blocks is crucial for a subset of the tasks we train on.
- Even if a block is not crucial for a task, we observe that adding it never hurts performance and in most cases improves it on all tasks. 
- We also show that tasks with less data benefit largely from joint training with other tasks, while performance on large tasks degrades only slightly if at all.
- A model has been given a host of tasks in parallel (language as well as visual tasks!).

### General description

[Ruder: Multi task learning](https://arxiv.org/abs/1706.05098)
Or more "friendly" description [here](http://ruder.io/multi-task-learning-nlp/)

### Limitations

We have to consider, that this question is in strong connection with generalization, albeit over the train-test split or even the "new", "live" data.

As such, it is also subject of the:

**[No free lunch theorem](https://en.wikipedia.org/wiki/No_free_lunch_theorem)**

_"states that any two optimization algorithms are equivalent when their performance is averaged across all possible problems"_

There is no universal model, not even a human one. **There will always be some things we don't know.**


## Outlook

It is obvious, that transfer learning - because of great emphasis on generalization -, and especially multi-task learning is also on the forefront of development towards general AI models.

A recently published amazing example proposes to use VGG CNN-s, pre-trained on standard ImageNet dataset for processing of sound spectrograms with remarkable efficiency.  A material presenting state of the art toolkits can be found here: [Björn Schuller: Deep Learning for signal analysis, DeepLearn2018 summer school, Genova](https://drive.google.com/file/d/1GCt1PAISMH-4L7fGuSvp_4DbzIiSXrZk/view?usp=sharing) [paper](https://www.isca-speech.org/archive/Interspeech_2017/pdfs/0434.PDF) and [toolkit](https://github.com/DeepSpectrum/DeepSpectrum). This shows that we still have to be rather humble, and we still have much to learn about invariances across domains.



# Foundational models and the new paradigm

With the advent of huge pre-trained transformers, some paradigmatic shifts happened. 

The core observation is, that __orders of magnitude more data and larger networks give consistently better performance__ on various datasets in case of transformers. (We don't know if other type of models would not have scaled also in this way, since for practical limitations we did not try.)

<img src="http://drive.google.com/uc?export=view&id=1K4cRzFEy5F4vjzp1RSo3iAdVn7L1cIf8" width=70%>

[source](https://www.lesswrong.com/tag/scaling-laws)

This observation led to a __huge boost in models sizes__, making it particularly infeasible for any but the biggest specialist organizations to train state of the art models.

<img src="http://drive.google.com/uc?export=view&id=1JqT7mpTOdUk7Q46ayMIgSnmrS78kWN9T" width=70%>

[source](https://www.lesswrong.com/posts/GzoWcYibWYwJva8aL/parameter-counts-in-machine-learning)

With this, another discovery coincided: In the original GPT-3 paper ["Language models are Few-Shot Learnerns"](https://arxiv.org/abs/2005.14165) the authors argue, that such a huge language model already learnt to solve many language based tasks in general, so it can be - without any tuning - utilized to solve novel problems that are communicated to it via it's inputs considered as __task defining prompts__.

<img src="http://drive.google.com/uc?export=view&id=1aZiFfAjUbSFdVkD8qbapkGoCf8cjzcbM" width=70%>

In this sense, the "pre-train and finetune" paradigm gave way to "prompt engineering".

See [this](https://thegradient.pub/prompting/) reference and the embedded talk about prompt engineering.

In [1]:
%%HTML 
<iframe width="560" height="315" src="https://www.youtube.com/embed/8GF8L5Vn7VE" ></iframe>


This lead some researchers to claim, that these huge pre-trained models are different qualitatively from a task specific solution, they can be considered [Foundational models](https://arxiv.org/abs/2108.07258), that are the basis of solving multiple tasks. Though this notion was hotly debated, some findings, like demonstrated in the paper [A Universal Law of Robustness via Isoperimetry](https://arxiv.org/abs/2105.12806) suggest, that we so far underestimated the parameter count for __smoothly interpolating__ a dataset, thus, there is theoretical justification for the application of large scale models, since their generalization properties are following this result strictly better.

As [this](https://www.reddit.com/r/mlscaling/comments/uznkhw/gpt3_2nd_anniversary/iab8vy2/?context=3) excellent discussion thread summarizes, this created a radically new situation in the ML space, since it shifts the power balance in favour of organizations that are capabel - from the resource as well as talent side - to train and own foundaitonal models, because they can potentially become API "providers" for others, who are just "consuming" the model's services.

This can cause huge imbalance in the economic dynamics of the field.

Nothign is decided yet, but it seems like, large scale training is here to stay...